## Result of GAiA's neural network

In [5]:
from model.GAiA_network import coeff_determination
from lib.dataset_utils import encode_position, read_many_hdf5
from lib.score_getter import ScoreGetter, Engine

import chess
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import platform
if platform.system() == "Darwin":
    %config InlineBackend.figure_format="retina"  # For high DPI display

We retrieve the model trained in the `train_model` notebook.

In [2]:
model = tf.keras.models.load_model("model/GAiA_Network")

2022-01-09 22:24:38.867694: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:24:38.898618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:24:38.898800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-09 22:24:38.899359: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
def plot_regression_results(model, X_test, y_test, path=None):
  preds = model.predict(X_test)
  r2_score = coeff_determination(tf.convert_to_tensor(y_test), preds).numpy()
  ma = np.max(y_test)
  mi = np.min(y_test)
  x = np.linspace(ma, mi, 100)
  plt.figure(figsize=(10,5))
  plt.plot(x, x, "r-.", label = "Predictions = True values")
  plt.scatter(y_test, preds, label = f"$R^2 = {r2_score:.2f}$")

  mask1 = y_test < -5000
  mask2 = preds > -3000
  mask3 = y_test > 5000
  mask4 = preds < 3000
  mask = (mask1 & mask2) | (mask3 & mask4)
  mask1 = scores_test > -2500
  mask2 = scores_test < 2500
  mask3 = preds > 5000
  mask4 = preds < -5000
  mask = mask | ((mask1 & mask2) & (mask3 | mask4))


  plt.scatter(y_test[mask], preds[mask], color="orange", label = "Misplaced points")

  plt.xlabel("True values (centipawn)")
  plt.ylabel("Predictions (centipawn)")
  plt.title(f"Results of GAiA's neural network with {X_test.shape[0]} test positions")
  plt.legend()
  if path:
    plt.savefig(path)

We load our test dataset and we plot the results (finally).

In [5]:
directory = "/media/gaetan/HDD/IA/Chess/Datasets/SE_ResNet/"
positions_test, scores_test = read_many_hdf5(397082, directory, "_test")
plot_regression_results(model, positions_test, scores_test, path="results/result.pdf")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/media/gaetan/HDD/IA/Chess/Datasets/SE_ResNet/397082_position_test.h5', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

In [3]:
def test_position (model, engine_path, fen):
  score_getter = ScoreGetter(engine_path, Engine.STOCKFISH)

  print(fen)
  board = chess.Board(fen)
  e = np.expand_dims(encode_position(board), axis=0)
  print(f"Score of Stockfish 14: {score_getter.get_score(fen)}")
  print(f"Score of GAiA: {int(model.predict(e).flatten()[0])}")

  return board

We observe the difference between GAiA and the Stockfish evaluation function on any chess position.

In [4]:
engine_path = "/home/gaetan/Téléchargements/stockfish"
test_position(model, engine_path, "r1bqkb1r/pppp1ppp/2nn4/1B2N3/8/8/PPPP1PPP/RNBQR1K1 b kq - 0 6")

r1bqkb1r/pppp1ppp/2nn4/1B2N3/8/8/PPPP1PPP/RNBQR1K1 b kq - 0 6
Score of Stockfish 14: 67.0


2022-01-09 22:24:44.172553: E tensorflow/stream_executor/cuda/cuda_dnn.cc:371] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2022-01-09 22:24:44.172627: E tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Possibly insufficient driver version: 495.29.5
2022-01-09 22:24:44.172660: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNKNOWN: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node GAiA/conv2d_8/Conv2D
 (defined at /home/gaetan/.local/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]] [Op:__inference_predict_function_1099]

Errors may have originated from an input operation.
Input Source operations connected to node GAiA/conv2d_8/Conv2D:
In[0] IteratorGetNext (defined at /home/gaetan/.local/lib/python3.9/site-packages/keras/engine/training.py:1610)	
In[1] GAiA/conv2d_8/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/gaetan/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>

In [ ]:
tf.saved_model.save(model, "test")

INFO:tensorflow:Assets written to: test/assets


In [ ]:
model.summary()